### Cuidados!
 
1. Whatsapp não gosta de nenhum tipo de automação
2. Isso pode dar merda, já to avisando
3. Isso não é o uso da API oficial do Whatsapp, o próprio whatsapp tem uma API oficial. Se o seu objetivo é fazer envio em massa ou criar aqueles robozinhos que respondem automaticamente no whatsapp, então use a API oficial
4. Meu objetivo é 100% educacional
5. O Xpath pode se atualizar.

### Dito isso, bora automatizar o envio de mensagens no Whatsapp

- Vamos usar o Selenium (vídeo da configuração na descrição)
- Temos 1 alternativa muito boa:
- Usar o https://web.whatsapp.com/send?phone=numero&text=texto (mais fácil, mais seguro, mas mais demorado)


In [41]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import os


servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)
navegador.get("https://web.whatsapp.com")


# esperar a tela do whatsapp carregar
while len(navegador.find_elements(By.ID,'side')) < 1: #o retorno é uma lista = se for vazio, não se cheio o elemento que diz se a tela carregou
    time.sleep(1)
time.sleep(2)

In [42]:
# o whatzap já carregou
import pandas as pd

tabela = pd.read_excel("Envios.xlsx")
display(tabela)

,nome,mensagem,arquivo,telefone
0,Deborinha,Este robozinho é legal.,robozinho-toto.gif,(34)999909660
1,Carlos,Este robozinho é legal.,robozinho-toto.gif,(34)999909660
2,Iara,Este robozinho é legal.,robozinho-toto.gif,(34)999909660
3,Neil Mestre,Este robozinho é legal.,robozinho-toto.gif,(34)999909660


In [43]:
import urllib

for linha in tabela.index:
    # enviar uma mensagem para a pessoa
    nome = tabela.loc[linha,"nome"]
    mensagem = tabela.loc[linha,"mensagem"]
    arquivo = tabela.loc[linha,"arquivo"]
    telefone = tabela.loc[linha,"telefone"]

    texto = mensagem.replace("fulano", nome)
    texto = urllib.parse.quote(texto)
    
    # enviar a mensagem
    link = f"https://web.whatsapp.com/send?phone={telefone}&text={texto}"
    
    navegador.get(link)
    
    # esperar a tela do whatsapp carregar >>> esperar um elemento que só existe na tela carregada.
    while len(navegador.find_elements(By.ID,'side')) < 1: #o retorno é uma lista = se for vazio, não se cheio o elemento que diz se a tela carregou
        time.sleep(1)
    time.sleep(2) # so de garantia
    
    # verificar se o numero do whatzap é valido
    if len(navegador.find_elements(By.XPATH, '//*[@id="app"]/div/span[2]/div/span/div/div/div/div/div/div[1]')) < 1:
        # enviar a mensagem
        navegador.find_element(By.XPATH, '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[2]/button/span').click()
        
        if arquivo != "N":
            caminho_completo = os.path.abspath(f"arquivos/{arquivo}")
            # anexar arquivo - clica no clips para ver opções
            navegador.find_element(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div/div/span').click()
            # procurar arquivo a ser anexo - anexa o arquivo
            navegador.find_element(By.XPATH, '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div/span/div/div/ul/li[4]/button/input').send_keys(caminho_completo)        
            time.sleep(2)
            # enviar aquivo anexado.
            navegador.find_element(By.XPATH,'//*[@id="app"]/div/div/div[2]/div[2]/span/div/span/div/div/div[2]/div/div[2]/div[2]/div/div').click()
    
    
    time.sleep(2)
